In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

# Завантаження даних Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
print("x_train original shape", x_train.shape)
print("y_train original shape", y_train.shape)

classes = ['футболка', 'брюки', 'свитер', 'платье', 'пальто', 'туфли', 'рубашка', 'кроссовки', 'сумка', 'ботинки']

plt.figure(figsize=(10, 10))
for i in range(100, 150):
    plt.subplot(5, 10, i - 100 + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(classes[y_train[i]])

# Преобразовання розмірності зображень
x_train = x_train.reshape(60000, 28, 28)
x_test = x_test.reshape(10000, 28, 28)
# Нормалізація даних
x_train = x_train / 255
x_test = x_test / 255

# Побудова моделі без Conv2D шару
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(units=800, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.5))
model.add(Dense(units=10, activation='softmax'))

# Компіляція моделі
model.compile(optimizer=SGD(learning_rate=0.01),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

# Посмотрим на архітектуру мережі
model.summary()

# Навчання моделі
history = model.fit(x_train, y_train,
                    batch_size=450,
                    epochs=50,
                    validation_split=0.2,
                    verbose=1)

# Збереження моделі
model.save('fashion_mnist_dense.h5')

# Оцінка якості навчання
scores = model.evaluate(x_test, y_test, verbose=1)
print("Доля верних ответов на тестовых данных, в процентах:", round(scores[1] * 100, 2))

# Распознавання одягу
n_rec = 496
plt.imshow(x_test[n_rec], cmap=plt.cm.binary)
plt.show()

# Підготовка зображення для розпізнавання
x = x_test[n_rec].reshape(1, 28, 28)  # Повернули форму вхідного зображення до (1, 28, 28)

# Розпізнавання
prediction = model.predict(x)

predicted_class = tf.argmax(prediction, axis=1).numpy()[0]
print("Номер класса:", predicted_class)
print("Название класса:", classes[predicted_class])


x_train original shape (60000, 28, 28)
y_train original shape (60000,)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 800)               628000    
                                                                 
 batch_normalization_2 (Bat  (None, 800)               3200      
 chNormalization)                                                
                                                                 
 dropout_2 (Dropout)         (None, 800)               0         
                                                                 
 dense_5 (Dense)             (None, 10)                8010      
                                                                 
Total params: 639210 (2.44 MB)
Trainable params: 